In [2]:
pip install fastparquet


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# Data Extraction for ALBUMIN: FRIEND OR FOE?

This script is in charge of extracting the data from the project developed during the tarragona 2022 datathon. In this code we study the variables that we want to extract from the icusics-db database and finally we extract and merge them to create a final dataset.  


In [360]:
import boto3
import pandas as pd
import numpy as np
import re
from sagemaker import get_execution_role
role = get_execution_role()
s3 = boto3.resource('s3')

First of all we load the patients database and the d_variables database and d_pharma in order to get the whole cohort of patients possible and the reference of the variables. 

In [361]:
icusics_db_patients = pd.read_parquet("s3://icusics-db/patients/patients.parquet")

In [174]:
icusics_db_patients

,hospital_coded,a_patientid,patientsex,age,height,weight,bmi,hospadmtime,distime,hospdistime,hospital_outcome
0,1,1000091,M,70,160,50,20,-549,6034,24482,ALIVE
1,1,1000109,M,50,160,80,31,-12516,10118,32875,ALIVE
2,1,1000211,M,70,160,60,23,-13013,2495,2495,EXITUS
3,1,1000999,F,30,150,60,27,-1731,8597,21402,ALIVE
4,1,1001000,F,50,160,80,31,0,31713,31830,EXITUS
...,...,...,...,...,...,...,...,...,...,...,...
21134,6,6998109,M,50,180,70,22,-17,1756,8806,ALIVE
21135,6,6998382,F,50,160,100,39,-1216,4321,8410,ALIVE
21136,6,6998565,F,40,170,90,31,-680,3436,8126,ALIVE
21137,6,6998846,F,80,160,80,31,-553,1267,2610,ALIVE


In [370]:
d_variables = pd.read_parquet('s3://icusics-db/d_variables/d_variables.parquet')

In [373]:
key_chars = 'apache'

apache = d_variables[  
    (d_variables['name'].str.contains(key_chars, case=False, na=False)) | (
    d_variables['description'].str.contains(key_chars, case=False, na=False)) | (
    d_variables['choicestringvalue'].str.contains(key_chars, case=False, na=False))]

apache

,hospital_coded,table,a_variableid,choicecode,choicestringvalue,name,abbreviation,description
0,1,derived_numeric,1030000100,NaN,None,Chronic health evaluation,CHE,In APACHE II and A2
2,1,derived_numeric,1030000140,NaN,None,Highest 24 h APACHE II,APACHE II,Automatically calculated APACHE II score
15,1,derived_numeric,1030000350,NaN,None,APACHE 2 validated,APACHE2vàlid,Validated APACHE II score
16,1,derived_numeric,1030000360,NaN,None,APACHE 2 risk of hosp death,R(APACHE2),Validated R(APACHE II) score
17,1,derived_numeric,1030000370,NaN,None,APACHE 2 daily worst APS,A2 APS,Validated APS score
...,...,...,...,...,...,...,...,...
27102,6,observed_categoric,6014000150,0.0,No intervingut,Intervingut a l'ingrés,Intervingut?,Detalls admissió: Nou pacient. Tipus d'ingrés....
27103,6,observed_categoric,6014000150,1.0,Quirúrgic urgent,Intervingut a l'ingrés,Intervingut?,Detalls admissió: Nou pacient. Tipus d'ingrés....
27104,6,observed_categoric,6014000150,2.0,Quirúrgic Programat,Intervingut a l'ingrés,Intervingut?,Detalls admissió: Nou pacient. Tipus d'ingrés....
27105,6,observed_categoric,6014000150,3.0,None,Intervingut a l'ingrés,Intervingut?,Detalls admissió: Nou pacient. Tipus d'ingrés....


In our study, the albumin is one of the main variables, as it will set the time 0. Therefore, we need to extract this variable. 
As an example on how to get the variables_id for the extraction of the data, we have use the albumin. 

In [374]:
d_pharma = pd.read_parquet('s3://icusics-db/d_pharma/d_pharma.parquet')

In [375]:
key_chars = 'ALBUMINA'
d_pharma = d_pharma[ (
    (d_pharma['pharmaname'].str.contains(key_chars, case=False, na=False)) ) ]
d_pharma

,hospital_coded,a_pharmaid,pharmaname,pharmagroupname,pharmaformunit,pharmadoseunit,pharmadoseformratio,pharmavolumeunit,pharmavolumeformratio
28,1,1001000391,ALBUMINA HUMANA 20% 10 g/50 ml vial,None,ml,g,0.20,ml,1.0
29,1,1001000392,ALBUMINA HUMANA 5% 25 g/500 ml vial,None,ml,g,0.05,ml,1.0
814,1,1001001938,ALBUMINA HUMANA 20% 10 g/50 ml vial PED,B05AA,ml,g,0.20,ml,1.0
867,1,1001002249,ALBUMINA HUMA 5% 25g/500ml PED,A Cardiovascular,ml,g,0.05,ml,1.0
1153,2,2001001043,ALBUMINA HUM.20% 50ML VIAL,None,g,ml,5.00,ml,5.0
1270,2,2001001287,ALBUMINA HUMANA 20% 10 g/50 ml vial,B05AA,ml,g,0.20,ml,1.0
1900,3,3000000065,ALBUMINA HUMANA 20% 10 g/50 ml vial,.Albúmina,ml,g,0.20,ml,1.0
1985,3,3000000163,ALBUMINA HUMANA 5% 25 g/500 ml vial,.Albúmina,ml,g,0.05,ml,1.0
2558,3,3000005306,ALBUMINA HUMANA 20%,.Substituts del plasma,ml,g,0.20,ml,1.0
3101,4,4001006392,ALBUMINA HUMANA 20%,Coloïds,ml,g,0.20,ml,1.0


We talk with clinicians and they decide if the ids are correct. 

In [376]:
albumin_ids = [1001000391,
1001000392,
1001001938,
1001002249,
2001001043,
2001001287,
3000000065,
3000000163,
3000005306,
4001006392,
4001006393,
4001008056,
5000000065,
5000005306,
6000000065,
6000005306]

# Albumin extraction

Now we have to query the pharma records table in order to extract the data for the albumin that the patients received. 

In [17]:
db = 'icusics-db'
hospitales = ["h1","h2","h3","h4","h5","h6"]
bucket = s3.Bucket(f'{db}')
lista_pats = tuple()
final_pharmaname = pd.DataFrame()

for my_bucket_object in bucket.objects.all(): # runs over all of the buckets
    
    if all(x in my_bucket_object.key for x in ['pharma_records/']): # checks if the are pharma records
        print(my_bucket_object.key)
        boundaries = re.findall('\d+', my_bucket_object.key)
        chunk = pd.read_parquet(f's3://{db}/{my_bucket_object.key}', engine="fastparquet")
        print(len(chunk))
        chunk = chunk[chunk.a_pharmaid.isin(albumin_ids)] # selects only those pharmacs that are albumin
        print(len(chunk))
        # chunk = chunk[chunk.a_patientid==patid]
        final_pharmaname= pd.concat([final_pharmaname,chunk])


pharma_records/pharma_records_h1.parquet
11862232
6268
pharma_records/pharma_records_h2.parquet
1628907
4579
pharma_records/pharma_records_h3.parquet
8273265
3739
pharma_records/pharma_records_h4.parquet
11401460
27142
pharma_records/pharma_records_h5.parquet
2331093
425
pharma_records/pharma_records_h6.parquet
8240802
5975


We check different numbers from the albumins extracted. 

In [37]:
final_pharmaname.routename.value_counts()

IV              20143
PIV FLUID       16839
PERF IV         10092
PIV              1045
PerfInterm          5
PERF IV CONT        3
NO ADMINISTR        1
Name: routename, dtype: int64

In [34]:
print("number of albumins "+str(len(final_pharmaname))+" number of patients "+str(len(set(tuple(final_pharmaname.a_patientid)))))

number of albumins 48128 number of patients 2409


Merge the albumin extracted with the patient database to check if we have from each hospital. 

In [124]:
cohort = final_pharmaname.merge(icusics_db_patients, how="left", on="a_patientid")

In [125]:
cohort[["a_patientid", "hospital_coded", "a_pharmaid"]].drop_duplicates("a_patientid").hospital_coded.value_counts()

4    666
1    609
2    419
6    402
3    253
5     60
Name: hospital_coded, dtype: int64

# Getting the whole cohort

We want to study differences between patients that received albumin and those who didnt. We have to get a group of control patients. 

In [126]:
cohort =cohort[["a_patientid", "hospital_coded", "a_pharmaid", "ordernumber", "time", "givendose", "routename"]]

We create a albumina dataframe. 

In [ ]:
cohort["albumina"] = 1
icusics_db_patients_with_albumine = icusics_db_patients.merge(cohort.drop_duplicates("a_patientid")[["a_patientid","albumina"]], on="a_patientid", how="left")


And the controls. We end up having subcohort with 9636 patients. 

In [ ]:
ids_sin_alb = list(set(tuple(icusics_db_patients_with_albumine[icusics_db_patients_with_albumine.albumina!=1].a_patientid)))
albumineros = icusics_db_patients_with_albumine[icusics_db_patients_with_albumine.albumina==1]
import random
lista_final = list()
for i in range(0,2409*3): # We get 3 times the albumin patients as controls
    duplicado=True
    while duplicado:
        a = random.randint(0,len(ids_sin_alb)) # generate random patient with no albumin
        if (ids_sin_alb[a] not in lista_final) & (ids_sin_alb[a] not in list(set(albumineros.a_patientid))): 
            lista_final.append(ids_sin_alb[a]) # append patient to the lst
            duplicado=False
subcohort_patients = icusics_db_patients_with_albumine[icusics_db_patients_with_albumine.a_patientid.isin(lista_final)]
subcohort = pd.concat([albumineros,subcohort_patients]) # COHORTE CON EL QUE TRABAJAMOS GROUPS BOTH
subcohort_pats = list(set(subcohort.a_patientid)) # List of patients

# Extraction of the data

First we have generated the extraction of all the variables. Then we will process them and merge them later. 

**WARNING DISCLOSURE**: At the time of writing this, we are NOT sure that you can extract ALL of the data without merging step by step. If something fails, merge everything until the step you are now. 

In [266]:
db = 'icusics-db'
hospitales = ["h1","h2","h3","h4","h5","h6"]
bucket = s3.Bucket(f'{db}')
lista_pats = tuple()
final_chunk = pd.DataFrame()

for my_bucket_object in bucket.objects.all():
    
    if all(x in my_bucket_object.key for x in ['diagnoses/']):
        boundaries = re.findall('\d+', my_bucket_object.key)
        chunk = pd.read_parquet(f's3://{db}/{my_bucket_object.key}', engine="fastparquet")
        # chunk = chunk[chunk.a_variableid.isin([3030000140])]
        # chunk = chunk[chunk.a_patientid==patid]
        final_chunk= pd.concat([final_chunk,chunk])


### Select those with cirrosis

In [321]:
diagnosticos_pivotar = final_chunk[ (final_chunk.a_patientid.isin(subcohort_pats))]

In [305]:
codigos_cortos=["571","305","789","572","K70","K74","K72","K76","F10"]
codigos_largos=["456.1","456.8","348.3","291.2","573.8","303.9","K71.7"]


In [322]:
diagnosticos_pivotar_l=diagnosticos_pivotar[diagnosticos_pivotar.referencecode.isin(codigos_largos)]

In [323]:
diagnosticos_pivotar["codigo"]=[elem[0:3] for elem in diagnosticos_pivotar.referencecode]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [324]:
diagnosticos_pivotar_c=diagnosticos_pivotar[diagnosticos_pivotar.codigo.isin(codigos_cortos)]

In [325]:
len(diagnosticos_pivotar_c),len(diagnosticos_pivotar_l)

(1193, 26)

In [328]:
cirrosis = pd.concat([diagnosticos_pivotar_l,diagnosticos_pivotar_c])
cirrosis = cirrosis.drop_duplicates("a_patientid")

In [329]:
cirrosis["Cirrosis"]=1

### Generate primary diagnost

In [154]:
diagnosticos_pivotar[diagnosticos_pivotar.referencecode.str.contains("A01")]

,a_patientid,diag_type,referencecode,referencecodename


In [140]:
diagnosticos_pivotar["codigo"]=[elem[0:3] for elem in diagnosticos_pivotar.referencecode]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [147]:
diagnosticos_pivotar

,a_patientid,diag_type,referencecode,referencecodename,codigo
7,1000109,primary,Ascites. Líquid a cavitat peritoneal:789,Ascites. Líquid a cavitat peritoneal,789
13,1001000,primary,"Empiema amb fístula. Fístula:broncocutània, br...","Empiema amb fístula. Fístula:broncocutània, br...",510
14,1001170,primary,Xoc sèptic:785,Xoc sèptic,785
15,1001170,secondary,Aplasia medul.lar secundaria:284,Aplasia medul.lar secundaria,284
16,1001170,secondary,Colecistitis gangrenosa:575,Colecistitis gangrenosa,575
...,...,...,...,...,...
6540,6991794,secondary,Defecte de la coagulació no especificat:D68,Defecte de la coagulació no especificat,D68
6541,6991794,secondary,Fallida hepàtica aguda i subaguda\t:K72,Fallida hepàtica aguda i subaguda\t,K72
6542,6991794,secondary,Fallida renal aguda:N17,Fallida renal aguda,N17
6543,6991794,secondary,Fallida respiratòria aguda:J96,Fallida respiratòria aguda,J96


### APACHE

In [256]:
key_chars = 'apache'

apache = d_variables[  (d_variables['hospital_coded']==2)& (
    (d_variables['name'].str.contains(key_chars, case=False, na=False)) | (
    d_variables['description'].str.contains(key_chars, case=False, na=False)) | (
    d_variables['choicestringvalue'].str.contains(key_chars, case=False, na=False))) ]

apache_ids = [1030000140,2030000140,3030000140,4030000140,5030000140,6030000140]


In [273]:
bucket = s3.Bucket(f'{db}')
lista_pacientes_con_apache = tuple()
final_apache = pd.DataFrame()
total = len(subcohort_pats) 
i = 0
for my_bucket_object in bucket.objects.all():
    
    if all(x in my_bucket_object.key for x in ['derived_numeric/']):
        boundaries = re.findall('\d+', my_bucket_object.key)
        ub = boundaries.pop(1)
        lb = boundaries.pop()
        
        for patid in subcohort_pats:
            if patid>=int(lb) and patid<=int(ub):
                print(i, end="\r")
                i+=1
                # print(patid,lb,ub)
                chunk = pd.read_parquet(f's3://{db}/{my_bucket_object.key}', engine="fastparquet")
                chunk = chunk[chunk.a_variableid.isin(apache_ids)]
                chunk = chunk[chunk.a_patientid==patid]
                final_apache= pd.concat([final_apache,chunk])


In [277]:
final_apache.to_csv("final_apache.csv")

## TCRR

In [3]:
tcrr =[1000001380,2000001380,3000001380,4000001380,5000001380,6000001380] # TCRR

In [ ]:
bucket = s3.Bucket(f'{db}')
tcrr = pd.DataFrame()
total = len(subcohort_pats)
i = 0
j=0
for my_bucket_object in bucket.objects.all():
    
    if all(x in my_bucket_object.key for x in ['monitored_numeric/']):
        boundaries = re.findall('\d+', my_bucket_object.key)
        ub = boundaries.pop(1)
        lb = boundaries.pop()
        
        for patid in subcohort_pats:
            if patid>=int(lb) and patid<=int(ub):
                print(round((i/total)*100), end="\r")
                i+=1
                # print(patid,lb,ub)
                
                chunk = pd.read_parquet(f's3://{db}/{my_bucket_object.key}', engine="fastparquet")
                chunk = chunk[chunk.a_variableid.isin(tcrr)]
                chunk = chunk[chunk.a_patientid==patid]
                tcrr= pd.concat([tcrr,chunk])


## Monitorizadas numericas

In [6]:
monitorizadas = [1000004000,2000004000,3000004000,4000004000,5000004000,6000004000, # saturacion oxigeno
                 1000000110,2000000110,3000000110,4000000110,5000000110,6000000110, # PAM 	
                 1000000610,2000000610,3000000610,4000000610,5000000610,6000000610, # PAM  NI
                 1000000200,2000000200,3000000200,4000000200,5000000200,6000000200, # HR
                ]

In [ ]:
db = 'icusics-db'
bucket = s3.Bucket(f'{db}')
lista_pacientes_con_apache = tuple()
final_mon = pd.DataFrame()
total = len(subcohort_pats)
i = 0
j=0
times = subcohort_with_albumine_and_cirrosis_and_apache[["a_patientid","time"]].rename({"time":"tiempo"},axis=1)
for my_bucket_object in bucket.objects.all():
    
    if all(x in my_bucket_object.key for x in ['monitored_numeric/']):
        boundaries = re.findall('\d+', my_bucket_object.key)
        ub = boundaries.pop(1)
        lb = boundaries.pop()
        
        for patid in subcohort_pats:
            if patid>=int(lb) and patid<=int(ub):
                print(round((i/total)*100), end="\r")
                i+=1
                # print(patid,lb,ub)
                
                chunk = pd.read_parquet(f's3://{db}/{my_bucket_object.key}', engine="fastparquet")
                chunk = chunk[chunk.a_variableid.isin(monitorizadas)]
                chunk = chunk[chunk.a_patientid==patid]
                chunk = chunk.merge(times, on="a_patientid", how="left")
                chunk["diferencia"] = chunk["tiempo"]-chunk["time"]
                chunk = chunk[(chunk.diferencia>=0) & (chunk.diferencia<=30)] # filter by time of the patient
                # print(chunk.shape)
                final_mon= pd.concat([final_mon,chunk])
                # intenta juntarlo con el time manuel
                # 
                
                if final_mon.shape[0] > 1000000:
                    final_mon.to_csv(f"final_mon_{j}.csv")
                    j+=1
                    final_mon = pd.DataFrame()

In [18]:
final_mon.to_csv("final_mon_real.csv")

## lab

Change the code down to extract lactates or creats.

In [236]:
lactatos_ids = [1024000570,
1024000862,
1024000880,
2024000485,
2024000737,
3024000658,
3024000660,
3024000704,
3024002000,
4024001486,
4024002002,
4015003773,
5024000658,
5024000660,
5024000704,
5024002000,
6024002630,
6024002631,
6024003988,
6024003989]

creats_ids = [1020000600, 2020000600, 3024000718, 4020000600, 4024001964, 5024000718, 6020000600]

In [239]:
db = 'icusics-db'
hospitales = ["h1","h2","h3","h4","h5","h6"]
bucket = s3.Bucket(f'{db}')
final_lacts = pd.DataFrame()

for my_bucket_object in bucket.objects.all():
    
    if all(x in my_bucket_object.key for x in ['labresults_numeric/']):
        chunk = pd.read_parquet(f's3://{db}/{my_bucket_object.key}', engine="fastparquet")
        chunk = chunk[chunk.a_variableid.isin(lactatos_ids)]
        
        final_lacts = pd.concat([final_lacts,chunk])

# MERGE DATA

### Merge with cirrosis.

In [ ]:
subcohort_with_albumine_and_cirrosis=subcohort_pats.merge(cirrosis[["a_patientid","Cirrosis"]], on="a_patientid", how="left")
diagnosticos_pivotar = diagnosticos_pivotar[diagnosticos_pivotar.diag_type=="primary"]
subcohort_with_albumine_and_cirrosis_and_diags=subcohort_with_albumine_and_cirrosis.merge(diagnosticos_pivotar[["a_patientid","codigo"]], on="a_patientid", how="left")

subcohort_with_albumine_and_cirrosis_and_diags

### Calculate time for each patient. 

For those that have albumin, we get their time of first albumin. For the control group, we generated a time between their ICU admission and their disward. 

In [345]:
# FIRST we generate a random time for each one.
time = list()
contador = 0
for pat in subcohort_with_albumine_and_cirrosis_and_diags.a_patientid:
    paciente = subcohort_with_albumine_and_cirrosis_and_diags[subcohort_with_albumine_and_cirrosis_and_diags.a_patientid==pat]
    try:
        time.append(random.randint(0,paciente["distime"][contador]))
    except: 
        time.append(random.randint(paciente["hospadmtime"][contador],paciente["distime"][contador]))
    contador+=1

In [373]:
subcohort_with_albumine_and_cirrosis_and_diags["time_random"] = time

Then we get the albumina time for the patients with albumin

In [370]:
tiempo_albumina = cohort.sort_values(["a_patientid","time"]).drop_duplicates("a_patientid").rename({"time":"time_alb"},axis=1)

In [375]:
subcohort_with_albumine_and_cirrosis_and_diags = subcohort_with_albumine_and_cirrosis_and_diags.merge(tiempo_albumina[["a_patientid","time_alb"]], on="a_patientid", how="left")

The time is the albumina time if the patients is not a control patient. 

In [376]:
subcohort_with_albumine_and_cirrosis_and_diags["time"] = np.where(subcohort_with_albumine_and_cirrosis_and_diags["time_alb"].notnull(),subcohort_with_albumine_and_cirrosis_and_diags["time_alb"],subcohort_with_albumine_and_cirrosis_and_diags["time_random"]) 

### ADD APACHE

We select the first apache of each patient and add it to the dataset. 

In [ ]:
apache_sin_dups = final_apache.sort_values(["a_patientid","time"]).drop_duplicates("a_patientid")
subcohort_with_albumine_and_cirrosis_and_diags_and_apache = subcohort_with_albumine_and_cirrosis_and_diags.merge(apache_sin_dups[["a_patientid","value"]], on="a_patientid", how="left")

### ADD TCRR

For TCRR we want to make 2 things. First of all, we want to delete those patients that had tcrr BEFORE albumin intake.
Second of all, we want to get if a patient had tcrr 3 days AFTER receiving albumin. 

In [128]:
tcrr = tcrr.rename({"time":"tiempo_tcrr", "value":"valor_tcrr"},axis=1)

In [129]:
subcohortv2_tcrr = subcohort_with_albumine_and_cirrosis_and_diags_and_apache.merge(tcrr.drop("a_variableid", axis=1), on="a_patientid", how="left")

Calculate the difference between albumin time and tcrr time. 
IF a patient has a diferencia de tiempo tcrr OVER 0, he had TCRR before albumin and this patient will be deleted. 

In [130]:
subcohortv2_tcrr["diferencia_tiempo_tcrr"] = subcohortv2_tcrr["time"] - subcohortv2_tcrr["tiempo_tcrr"]
subcohortv2_tcrr["eliminar_por_tcrr_before"] = np.where(subcohortv2_tcrr["diferencia_tiempo_tcrr"]>0,1,0)

In [131]:
pacientes_a_eliminar_por_tcrr_antes = subcohortv2_tcrr.sort_values(["a_patientid","diferencia_tiempo_tcrr"], ascending=[True, False]).drop_duplicates("a_patientid")

In [132]:
pacientes_a_eliminar_por_tcrr_antes.eliminar_por_tcrr_before.value_counts()

0    9261
1     375
Name: eliminar_por_tcrr_before, dtype: int64

We will delete 375 pats

In [133]:
pacientes_a_eliminar_por_tcrr_antes = list(set(pacientes_a_eliminar_por_tcrr_antes[pacientes_a_eliminar_por_tcrr_antes.eliminar_por_tcrr_before==1].a_patientid))

If a patient received tcrr between 1st and 3rd day after albumin, we flag this as a variable. 

In [134]:
subcohortv2_tcrr = subcohortv2_tcrr.sort_values(["a_patientid","diferencia_tiempo_tcrr"], ascending=[True, False])
subcohortv2_tcrr = subcohortv2_tcrr[(subcohortv2_tcrr.diferencia_tiempo_tcrr<=-1440) & (subcohortv2_tcrr.diferencia_tiempo_tcrr>=-4320)]
subcohortv2_tcrr = subcohortv2_tcrr.drop_duplicates("a_patientid")

In [135]:
subcohortv2_tcrr["tcrr"] = 1
subcohortv2_tcrr["tcrr"].value_counts()

1    424
Name: tcrr, dtype: int64

In [175]:
subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr = subcohort_with_albumine_and_cirrosis_and_diags_and_apache[~subcohort_with_albumine_and_cirrosis_and_diags_and_apache.a_patientid.isin(pacientes_a_eliminar_por_tcrr_antes)]
subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr = subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr.merge(subcohortv2_tcrr[["a_patientid","tcrr"]], on="a_patientid", how="left")

### ADD CREAT

For creatinine, we want to obtain the last creatinine before albumine administration, the first creatinine after 3 days of albumin administration and the first creatinine the patient had. 

In [ ]:
creats_values = pd.read_csv("creats_values.csv",index_col=0)
creats_values = creats_values.rename({"time":"tiempo_creat", "value":"valor_creat"},axis=1)

In [ ]:
subcohortv2_creat = subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr.merge(creats_values.drop("a_variableid", axis=1), on="a_patientid", how="left")

Last creatinine before. 

In [ ]:
subcohortv2_creat["diferencia_tiempo_creat"] = subcohortv2_creat["time"] - subcohortv2_creat["tiempo_creat"]
subcohortv2_creat = subcohortv2_creat.sort_values(["a_patientid","diferencia_tiempo_creat"], ascending=[True, True])
subcohortv2_creat_before = subcohortv2_creat[(subcohortv2_creat.diferencia_tiempo_creat>=0)]
subcohortv2_creat_before = subcohortv2_creat_before.drop_duplicates("a_patientid")
subcohortv2_creat_before = subcohortv2_creat_before.rename({"valor_creat":"creat_before"},axis=1).drop("diferencia_tiempo_creat",axis=1)

First creatinine after. 

In [ ]:
subcohortv2_creat["diferencia_tiempo_creat"] = subcohortv2_creat["time"] - subcohortv2_creat["tiempo_creat"]
subcohortv2_creat = subcohortv2_creat.sort_values(["a_patientid","diferencia_tiempo_creat"], ascending=[True, False])
subcohortv2_creat_later = subcohortv2_creat[(subcohortv2_creat.diferencia_tiempo_creat<=-720)]
subcohortv2_creat_later = subcohortv2_creat_later.drop_duplicates("a_patientid")
subcohortv2_creat_later = subcohortv2_creat_later.rename({"valor_creat":"creat_later"},axis=1).drop("diferencia_tiempo_creat",axis=1)

First creatinine. 

In [ ]:
subcohortv2_creat["diferencia_tiempo_creat"] = subcohortv2_creat["time"] - subcohortv2_creat["tiempo_creat"]
subcohortv2_creat = subcohortv2_creat.sort_values(["a_patientid","diferencia_tiempo_creat"], ascending=[True, False])
primera_creat = subcohortv2_creat.sort_values(["a_patientid","tiempo_creat"]).drop_duplicates("a_patientid").rename({"valor_creat":"first_creat"},axis=1)

In [ ]:
subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr_and_creat = subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr.merge(subcohortv2_creat_before[["a_patientid","creat_before"]], on="a_patientid", how="left")
subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr_and_creat = subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr_and_creat.merge(subcohortv2_creat_later[["a_patientid","creat_later"]], on="a_patientid", how="left")
subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr_and_creat = subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr_and_creat.merge(primera_creat[["a_patientid","first_creat"]], on="a_patientid", how="left")

### ADD LACTATE

For the lactate, we want to get the lactate between 12 hours and the moment of albumin administration. So we can get the state of the patient before albumin. 

In [244]:
# LACTATO
lacts_pats = lacts_pats.rename({"time":"tiempo_lacts", "value":"valor_lacts"},axis=1)
subcohortv2_lacts = subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr_and_creat.merge(lacts_pats.drop("a_variableid", axis=1), on="a_patientid", how="left")

In [250]:
subcohortv2_lacts["diferencia_tiempo_lact"] = subcohortv2_lacts["time"] - subcohortv2_lacts["tiempo_lacts"]
subcohortv2_lacts = subcohortv2_lacts.sort_values(["a_patientid","diferencia_tiempo_lact"], ascending=[True, True])
subcohortv2_lact_before = subcohortv2_lacts[(subcohortv2_lacts.diferencia_tiempo_lact>=0) & (subcohortv2_lacts.diferencia_tiempo_lact<=720)] # filter by name
subcohortv2_lact_before = subcohortv2_lact_before.drop_duplicates("a_patientid")
subcohortv2_lact_before = subcohortv2_lact_before.rename({"valor_lacts":"lact_before"},axis=1).drop("diferencia_tiempo_lact",axis=1)
subcohortv2_lact_before

,hospital_coded,a_patientid,patientsex,age,height,weight,bmi,hospadmtime,distime,hospdistime,...,tcrr,creat_before,creat_later,first_creat,PAM,HR,PAM_NI,SatO2,tiempo_lacts,lact_before
1,1,1000091,M,70,160,50,20,-549,6034,24482,...,NaN,0.86,NaN,0.86,NaN,98.0,70.0,98.0,1385.0,1.1
5,1,1000109,M,50,160,80,31,-12516,10118,32875,...,NaN,1.07,0.63,1.28,70.5,74.0,NaN,100.0,1211.0,1.4
67,1,1002069,F,70,160,60,23,-558,73410,74241,...,1.0,NaN,0.56,1.36,NaN,NaN,NaN,NaN,-76.0,2.6
163,1,1002286,F,70,160,60,23,-2290,86710,87245,...,NaN,0.76,0.80,0.72,86.0,70.0,NaN,100.0,9612.0,2.1
209,1,1002635,M,80,160,60,23,0,5267,5984,...,NaN,1.50,NaN,1.23,71.0,60.0,NaN,98.5,4594.0,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112491,6,6990826,M,50,170,90,31,0,986,6584,...,NaN,0.61,NaN,0.36,NaN,91.0,89.0,99.0,147.0,0.9
112498,6,6991794,F,80,160,50,20,-1,11390,24101,...,NaN,2.03,NaN,1.78,NaN,79.0,NaN,96.0,6431.0,0.6
112504,6,6993396,M,70,160,70,27,-2,2628,9682,...,NaN,0.91,NaN,1.46,NaN,91.0,92.0,98.0,1254.0,3.2
112514,6,6998846,F,80,160,80,31,-553,1267,2610,...,NaN,0.75,NaN,0.82,NaN,71.0,NaN,96.0,859.0,1.1


In [ ]:
subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr_and_creat_and_lact = subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr_and_creat.merge(subcohortv2_lact_before[["a_patientid","lact_before"]], on="a_patientid", how="left")

### ADD Monitorized variables 

We have extracted PAM, PAM NI, SatO2 and HR. We want the grouped value for each patient 30 mins before the alb administration. 

In [229]:
final_mon_real = pd.read_csv("final_mon_real.csv",index_col=0)

final_mon_real = final_mon_real.groupby(["a_patientid","a_variableid"]).agg({"value":"median"}).reset_index() # group by patient and variable
final_mon_real["a_variableid"] = final_mon_real["a_variableid"].astype("str")
final_mon_real["a_variableid"] = final_mon_real["a_variableid"].str[2:] # simplify the ids so we can merge them together even if they come from different hospitals.
final_mon_real = final_mon_real.pivot(index='a_patientid', columns="a_variableid",values="value").reset_index() # pivot table for a file by patient
final_mon_real= final_mon_real.rename({"00000110":"PAM","00000200":"HR","00000610":"PAM_NI","00004000":"SatO2"},axis=1)


In [230]:
subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr_and_creat_and_lact_and_mons = subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr_and_creat_and_lact.merge(final_mon_real[["a_patientid","PAM","HR","PAM_NI","SatO2"]], on="a_patientid", how="left")

In [ ]:
subcohort_with_albumine_and_cirrosis_and_diags_and_apache_and_tcrr_and_creat_and_lact_and_mons.to_csv("final_db.csv")